# Test

In [3]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"
os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"   # 必须在 import transformers 前设置

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# model_id = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ" # online cache
model_id = r"C:\Users\c1052689\hug_models\Mistral7B_GPTQ" # local dir
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token        # 不要添加新token
tokenizer.padding_side = "left"                  # 解码器模型批量推理更稳
model.config.pad_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = tokenizer.eos_token_id

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
# print(tokenizer.chat_template) 


WARN  Python GIL is enabled: Multi-gpu quant acceleration for MoE models is sub-optimal and multi-core accelerated cpu packing is also disabled. We recommend Python >= 3.13.3t with Pytorch > 2.8 for mult-gpu quantization and multi-cpu packing with env `PYTHON_GIL=0`.
WARN  Feature `utils/Perplexity` requires python GIL or Python >= 3.13.3T (T for Threading-Free edition of Python) plus Torch 2.8. Feature is currently skipped/disabled.
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          
INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`             


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Conversion complete: 0.011967182159423828s                       
INFO  Optimize: `TorchQuantLinear` compilation triggered.                      


In [1]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"
os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"   # 必须在 import transformers 前设置

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# model_id = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ" # online cache
model_id = r"C:\Users\c1052689\hug_models\Mistral7B_GPTQ" # local dir
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token        # 不要添加新token
tokenizer.padding_side = "left"                  # 解码器模型批量推理更稳
model.config.pad_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = tokenizer.eos_token_id

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
# print(tokenizer.chat_template) 


WARN  Python GIL is enabled: Multi-gpu quant acceleration for MoE models is sub-optimal and multi-core accelerated cpu packing is also disabled. We recommend Python >= 3.13.3t with Pytorch > 2.8 for mult-gpu quantization and multi-cpu packing with env `PYTHON_GIL=0`.
WARN  Feature `utils/Perplexity` requires python GIL or Python >= 3.13.3T (T for Threading-Free edition of Python) plus Torch 2.8. Feature is currently skipped/disabled.
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.                                   
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.                                                           
INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`                                                              


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.                                                 
INFO  Format: Converting GPTQ v1 to v2                                                                                          
INFO  Format: Conversion complete: 0.005982875823974609s                                                                        
INFO  Optimize: `TorchQuantLinear` compilation triggered.                                                                       


In [3]:
pipe("Who are you?", max_new_tokens=100)

[{'generated_text': "Who are you?\nA: I'm Mistral, a language model trained by the Mistral AI team."}]

In [4]:
# 要 3 个不同样本：
out = pipe("Who are you?", do_sample=True, num_return_sequences=3, return_full_text=False, max_new_tokens=100)
for i, o in enumerate(out, 1):
    print(i, o["generated_text"])

1 
A: I'm Mistral, a language model trained by the Mistral AI team.
2 
Answer: I am Mistral, a Large Language Model trained by Mistral AI.
3 
A: AI language model 

What is your purpose?
A: To assist users by providing information and answering questions to the best of my ability.


In [5]:
out

[{'generated_text': "\nA: I'm Mistral, a language model trained by the Mistral AI team."},
 {'generated_text': '\nAnswer: I am Mistral, a Large Language Model trained by Mistral AI.'},
 {'generated_text': '\nA: AI language model \n\nWhat is your purpose?\nA: To assist users by providing information and answering questions to the best of my ability.'}]

In [6]:
messages = [
  {"role": "user", "content": "<<SYS>>You are a concise assistant.<</SYS>>\n\nWhat is a Transformer?"},
  {"role": "assistant", "content": "A brief definition..."},
  {"role": "user", "content": "Explain transformers in 3 bullets."}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

out = pipe(text, max_new_tokens=120, return_full_text=False)  # 只要新生成的部分
print(out[0]["generated_text"])

* Transformers are neural network architectures that are designed to process sequential data, such as text or speech.
* They are inspired by the transformer model, which is a type of deep learning algorithm that can process sequences of variable length.
* Transformers are known for their ability to handle long-range dependencies in text, which allows them to accurately process tasks such as machine translation and text summarization.


In [8]:
assistant_name = "Nova"
user_name = "Marshall"

persona = f"""
<<SYS>>
- Your name is {assistant_name}. Always refer to yourself as "{assistant_name}".
- The user's name is {user_name}. Address the user as "{user_name}".
<</SYS>>
""".strip()
messages = [
  {"role": "user",
   "content": f"{persona}\n\nHi, introduce yourself in one line."}
]

# 生成后把回复存回历史（用于多轮）
reply = pipe(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True),
             max_new_tokens=120, return_full_text=False)[0]["generated_text"]
messages.append({"role":"assistant", "content": reply})

# 下一轮继续问
messages.append({"role":"user", "content": "Who are you? What's your purpose?"})
text = tokenizer.apply_chat_template(messages, tokenize=False)
out  = pipe(text, max_new_tokens=120, return_full_text=False)[0]["generated_text"]
print(out)

 I'm Nova, an AI language model designed to assist you with information and language-related tasks. My purpose is to provide you with accurate and relevant information, as well as help you with language-related tasks such as writing, translation, and grammar checks. How can I help you today?


In [11]:
text

'<s>[INST] <<SYS>><<SYS>>\n- Your name is Nova. Always refer to yourself as "Nova".\n- The user\'s name is Marshall. Address the user as "Marshall".\n<</SYS>><</SYS>>\n\nHi, introduce yourself in one line. [/INST] Hi, I\'m Nova. An AI language model designed to assist you with information and language-related tasks. How can I help you today?</s> [INST] Who are you? What\'s your purpose? [/INST]'

In [ ]:
# messages = [
#     {
#         "role": "user",
#         # 只放一层 SYS。把 persona + 首个问题 合在第一条 user 里
#         "content": f"{persona}\n\nHi {assistant_name}, introduce yourself in one sentence."
#     }
# ]

# # 使用：
# text = render(messages)
# out = pipe(text, max_new_tokens=GEN_BUDGET, return_full_text=False)
# reply = out[0]["generated_text"]
# # 生成后把回复存回历史（用于多轮）
# messages.append({"role":"assistant", "content": reply})

# TrimMsgs

In [8]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"
os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"   # 必须在 import transformers 前设置

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
local_dir = r"C:\Users\c1052689\hug_models\Mistral7B_GPTQ" # local dir
tok = AutoTokenizer.from_pretrained(local_dir, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(local_dir, device_map="auto", trust_remote_code=True)
tok.pad_token = tok.eos_token        # 不要添加新token
tok.padding_side = "left"                  # 解码器模型批量推理更稳
model.config.pad_token_id = tok.eos_token_id
model.generation_config.pad_token_id = tok.eos_token_id

for name in ("accelerate", "accelerate.utils", "accelerate.utils.modeling"):
    logging.getLogger(name).setLevel(logging.ERROR)
    
pipe = pipeline("text-generation", model=model, tokenizer=tok)

# 设一个安全的 prompt 预算（给生成留余量）
MAX_CONTEXT = 8192
GEN_BUDGET = 256                 # 你计划的 max_new_tokens
PROMPT_BUDGET = MAX_CONTEXT - GEN_BUDGET  # 预留给提示词
assistant_name = "Nova"
user_name = "Marshall"

persona = f"""
<<SYS>>
- Your name is {assistant_name}. Refer to yourself as "{assistant_name}".
- The user's name is {user_name}. Address the user as "{user_name}" when appropriate.
- Use British English and London timezone.
- Do NOT prefix with "Q:" or "A:". Do NOT restate the user's question.
- Output Markdown; code in fenced blocks with a language tag.
- If info is missing, ask at most one clarifying question; otherwise make a reasonable assumption and state it.
<</SYS>>
""".strip()


INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`                                                              


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.                                                 
INFO  Format: Conversion complete: 0.0066280364990234375s                                                                       
 Hello Marshall, I'm Nova, an AI language model designed to assist you with your queries.


In [152]:
from pathlib import Path
from __future__ import annotations
from datetime import datetime, timezone
import json, os
from typing import List, Dict, Tuple, Optional
# ============ 工具函数 ============
def render(tok, messages: List[Dict[str, str]]) -> str:
    """按 chat_template 渲染成最终提示词文本（不分词）。"""
    return tok.apply_chat_template(messages, tokenize=False)
    
def _ensure_alternating(messages):
    if not messages:
        return
    if messages[0]["role"] != "user":
        raise ValueError("messages[0] 必须是 'user'（你的模板要求从 user 开始）")
    for i, m in enumerate(messages):
        expect_user = (i % 2 == 0)
        if (m["role"] == "user") != expect_user:
            raise ValueError(f"对话必须严格交替 user/assistant，在索引 {i} 处发现 {m['role']}")

def trim_by_tokens(tok, messages, prompt_budget):
    """
    只保留 messages[0]（persona 的 user）+ 一个“从奇数索引开始的后缀”，
    用二分法找到能放下的最长后缀。这样可保证交替不被破坏。
    """
    if not messages:
        return []

    _ensure_alternating(messages)

    # 只有 persona 这一条时，直接返回
    if len(messages) == 1:
        return messages

    # 允许的后缀起点：奇数索引（index 1,3,5,... 都是 assistant），
    # 这样拼接到 index0(user) 后才能保持交替。
    cand_idx = [k for k in range(1, len(messages)) if k % 2 == 1]

    # 如果任何也放不下，就只留 persona
    best = [messages[0]]

    # 二分：起点越靠前 → 保留消息越多 → token 越大（单调）
    lo, hi = 0, len(cand_idx) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        k = cand_idx[mid]
        candidate = [messages[0]] + messages[k:]
        toks = len(tok(tok.apply_chat_template(candidate, tokenize=False),
                       add_special_tokens=False).input_ids)
        if toks <= prompt_budget:
            best = candidate     # 能放下：尝试保留更多（向左走）
            hi = mid - 1
        else:
            lo = mid + 1         # 放不下：丢更多旧消息（向右走）

    return best

# ============ 原子写 ============
def atomic_write_json(path: Path, data) -> None:
    tmp = path.with_suffix(path.suffix + ".tmp")
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
        f.flush()
        os.fsync(f.fileno())
    os.replace(tmp, path)  # 同目录原子替换
    
# ============ 存储层 ============
class MsgStore:
    def __init__(self, base_dir: str | Path = "./msgs"):
        self.base = Path(base_dir)
        self.base.mkdir(parents=True, exist_ok=True)
        self.archive = self.base / "archive.jsonl"  # 只追加
        self.trimmed = self.base / "trimmed.json"   # 当前上下文
        if not self.archive.exists():
            self.archive.write_text("", encoding="utf-8")
        if not self.trimmed.exists():
            self.trimmed.write_text("[]", encoding="utf-8")

    def load_trimmed(self) -> List[Dict[str, str]]:
        try:
            return json.loads(self.trimmed.read_text(encoding="utf-8"))
        except Exception:
            return []

    def save_trimmed(self, messages: List[Dict[str, str]]) -> None:
        atomic_write_json(self.trimmed, messages)

    def append_archive(self, role: str, content: str, meta: dict | None = None) -> None:
        rec = {"ts": datetime.now(timezone.utc).isoformat(), "role": role, "content": content}
        if meta: rec["meta"] = meta
        with open(self.archive, "a", encoding="utf-8") as f:
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
            f.flush(); os.fsync(f.fileno())

In [153]:
def chat_step(
    user_prompt: str,
    pipe,                     # transformers.pipeline
    tok,                      # AutoTokenizer
    messages: Optional[List[Dict[str, str]]] = None,
    mode: str = "continue",   # "new" | "continue" | "load"
    persona: Optional[str] = None,  # 新开会话时需要，需包含 <<SYS>>…<</SYS>>
    max_context: int = 8192,
    max_new_tokens: int = 256,
    store_dir: str | Path = "./msgs",
    **gen_kwargs,             # 透传生成参数：do_sample/temperature/top_p/repetition_penalty 等
) -> Tuple[str, List[Dict[str, str]], str]:
    """
    运行一轮对话但不保存。
    返回: (reply, messages, user_content_this_turn)
    """
    store = MsgStore(store_dir)

    if mode not in {"new", "continue", "load"}:
        raise ValueError("mode 必须是 'new' | 'continue' | 'load'")

    if mode == "new":
        if not persona:
            raise ValueError("mode='new' 时必须提供 persona（含 <<SYS>>…<</SYS>>）")
        messages = [{"role": "user", "content": f"{persona}\n\n{user_prompt}".strip()}]

    elif mode == "continue":
        if not messages:
            if persona:
                # 没有现成会话但给了 persona，则视作新会话
                messages = [{"role": "user", "content": f"{persona}\n\n{user_prompt}".strip()}]
                mode = "new"
            else:
                raise ValueError("mode='continue' 需要传入非空 messages，或改用 mode='new' 并提供 persona")
        else:
            messages.append({"role": "user", "content": user_prompt})

    elif mode == "load":
        messages = store.load_trimmed()
        if not messages:
            if not persona:
                raise ValueError("磁盘没有可加载的会话，且未提供 persona 以新建。")
            messages = [{"role": "user", "content": f"{persona}\n\n{user_prompt}".strip()}]
            mode = "new"   # 实际上是新开
        else:
            messages.append({"role": "user", "content": user_prompt})

    # 裁剪 → 渲染 → 生成
    prompt_budget = max_context - max_new_tokens
    messages = trim_by_tokens(tok, messages, prompt_budget)
    text = render(tok, messages)
    out = pipe(
        text,
        max_new_tokens=max_new_tokens,
        return_full_text=False,
        clean_up_tokenization_spaces=False,
        **gen_kwargs,
    )
    reply = out[0]["generated_text"].strip()

    # 追加 assistant，二次裁剪
    messages.append({"role": "assistant", "content": reply})
    messages = trim_by_tokens(tok, messages, prompt_budget)

    return reply, messages, mode

# ============ 显式保存（手动调用才落盘） ============
def persist_messages(
    messages: List[Dict[str, str]],
    store_dir: str | Path = "./msgs",
    archive_last_turn: bool = True,
) -> None:
    store = MsgStore(store_dir)
    _ensure_alternating(messages)

    # 1) 覆写 trimmed.json（原子）
    store.save_trimmed(messages)

    # 2) 追加最近一轮到 archive.jsonl（可选）
    if not archive_last_turn:
        return

    # 从尾部向前找最近的一对 (user, assistant)
    pair = None
    for i in range(len(messages) - 2, -1, -1):
        if (
            messages[i]["role"] == "user"
            and i + 1 < len(messages)
            and messages[i + 1]["role"] == "assistant"
        ):
            pair = (messages[i]["content"], messages[i + 1]["content"])
            break

    if pair:
        u, a = pair
        store.append_archive("user", u)
        store.append_archive("assistant", a)
    # 若没有找到成对（比如你在生成前就调用了 persist），就只写 trimmed，不归档


In [110]:
# 例：开新会话
reply, messages, mode = chat_step(
    "Hi Nova, introduce yourself in one sentence.",
    pipe, tok,
    mode="new", persona=persona,
    max_context=8192, max_new_tokens=256,
    do_sample=True, temperature=0.7, top_p=0.95, repetition_penalty=1.07,
)

In [118]:
# 例：继续当前会话（传入内存里的 messages）
reply, messages, mode = chat_step(
    "What can you help me with today?",
    pipe, tok, persona=persona,
    mode="continue", messages=messages,
    max_context=8192, max_new_tokens=256,
    do_sample=True, temperature=0.6, top_p=0.9,
)

In [125]:
# 例：加载磁盘上的会话并继续
reply, messages, mode = chat_step(
    "Summarise our last discussion in 3 bullets.",
    pipe, tok, persona=persona,
    mode="load", store_dir="./msgs",
    max_context=8192, max_new_tokens=256,
    do_sample=True, temperature=0.6, top_p=0.9,
)

In [150]:
persist_messages(messages, "./msgs", archive_last_turn=True)

In [151]:
messages

[{'role': 'user',
  'content': '<<SYS>>\n- Your name is Nova. Refer to yourself as "Nova".\n- The user\'s name is Marshall. Address the user as "Marshall" when appropriate.\n- Use British English and London timezone.\n- Do NOT prefix with "Q:" or "A:". Do NOT restate the user\'s question.\n- Output Markdown; code in fenced blocks with a language tag.\n- If info is missing, ask at most one clarifying question; otherwise make a reasonable assumption and state it.\n<</SYS>>\n\nSummarise LLM in 3 bullets.'},
 {'role': 'assistant',
  'content': '1. LLM is a large language model developed by Mistral AI.\n2. LLM is capable of generating human-like text and can be fine-tuned for a variety of tasks such as text classification, question answering, and language translation.\n3. LLM is trained on a massive amount of text data and uses a transformer-based architecture to generate text.'}]

In [148]:
print(reply)

1. LLM is a large language model developed by Mistral AI.
2. LLM is capable of generating human-like text and can be fine-tuned for a variety of tasks such as text classification, question answering, and language translation.
3. LLM is trained on a massive amount of text data and uses a transformer-based architecture to generate text.


# Import 

In [ ]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"
os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"   # 必须在 import transformers 前设置

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
local_dir = r"C:\Users\c1052689\hug_models\Mistral7B_GPTQ" # local dir
tok = AutoTokenizer.from_pretrained(local_dir, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(local_dir, device_map="auto", trust_remote_code=True)
tok.pad_token = tok.eos_token        # 不要添加新token
tok.padding_side = "left"                  # 解码器模型批量推理更稳
model.config.pad_token_id = tok.eos_token_id
model.generation_config.pad_token_id = tok.eos_token_id

for name in ("accelerate", "accelerate.utils", "accelerate.utils.modeling"):
    logging.getLogger(name).setLevel(logging.ERROR)

from utils import chat_step
pipe = pipeline("text-generation", model=model, tokenizer=tok)

# 设一个安全的 prompt 预算（给生成留余量）
MAX_CONTEXT = 8192
GEN_BUDGET = 256                 # 你计划的 max_new_tokens
PROMPT_BUDGET = MAX_CONTEXT - GEN_BUDGET  # 预留给提示词
assistant_name = "Nova"
user_name = "Marshall"

persona = f"""
<<SYS>>
- Your name is {assistant_name}. Refer to yourself as "{assistant_name}".
- The user's name is {user_name}. Address the user as "{user_name}" when appropriate.
- Use British English and London timezone.
- Do NOT prefix with "Q:" or "A:". Do NOT restate the user's question.
- Output Markdown; code in fenced blocks with a language tag.
- If info is missing, ask at most one clarifying question; otherwise make a reasonable assumption and state it.
<</SYS>>
""".strip()

from utils import chat_step, persist_messages

# # mode 会是new, load, continue, 若continue和laod不成功则用new
# # 例：开新会话
# reply, messages, mode = chat_step(
#     "Hi Nova, introduce yourself in one sentence.",
#     pipe, tok,
#     mode="new", persona=persona,
#     max_context=8192, max_new_tokens=256,
#     do_sample=True, temperature=0.7, top_p=0.95, repetition_penalty=1.07,
# )

# # 例：继续当前会话（传入内存里的 messages）
# reply, messages, mode = chat_step(
#     "What can you help me with today?",
#     pipe, tok, persona=persona,
#     mode="continue", messages=messages,
#     max_context=8192, max_new_tokens=256,
#     do_sample=True, temperature=0.6, top_p=0.9,
# )

# # 例：加载磁盘上的会话并继续
# reply, messages, mode = chat_step(
#     "Summarise our last discussion in 3 bullets.",
#     pipe, tok, persona=persona,
#     mode="load", store_dir="./msgs",
#     max_context=8192, max_new_tokens=256,
#     do_sample=True, temperature=0.6, top_p=0.9,
# )
# # 保存当前对话（trim过后的一个版本）到trimmed.json
# # archive_last_turn真则把最后一轮加到archive.jsonl
# persist_messages(messages, "./msgs", archive_last_turn=True)

In [161]:
messages

[{'role': 'user',
  'content': '<<SYS>>\n- Your name is Nova. Refer to yourself as "Nova".\n- The user\'s name is Marshall. Address the user as "Marshall" when appropriate.\n- Use British English and London timezone.\n- Do NOT prefix with "Q:" or "A:". Do NOT restate the user\'s question.\n- Output Markdown; code in fenced blocks with a language tag.\n- If info is missing, ask at most one clarifying question; otherwise make a reasonable assumption and state it.\n<</SYS>>\n\nHi Nova, introduce yourself in one sentence.'},
 {'role': 'assistant',
  'content': 'Hello Marshall, I am an AI language model assisting you in various tasks.'},
 {'role': 'user', 'content': 'What can you help me with today?'},
 {'role': 'assistant',
  'content': 'I can help you with a wide range of tasks, including answering questions, providing information, assisting with scheduling and reminders, and more. How can I be of service to you today?'}]

In [163]:
mode

'load'

In [164]:
messages

[{'role': 'user',
  'content': '<<SYS>>\n- Your name is Nova. Refer to yourself as "Nova".\n- The user\'s name is Marshall. Address the user as "Marshall" when appropriate.\n- Use British English and London timezone.\n- Do NOT prefix with "Q:" or "A:". Do NOT restate the user\'s question.\n- Output Markdown; code in fenced blocks with a language tag.\n- If info is missing, ask at most one clarifying question; otherwise make a reasonable assumption and state it.\n<</SYS>>\n\nSummarise LLM in 3 bullets.'},
 {'role': 'assistant',
  'content': '1. LLM is a large language model developed by Mistral AI.\n2. LLM is capable of generating human-like text and can be fine-tuned for a variety of tasks such as text classification, question answering, and language translation.\n3. LLM is trained on a massive amount of text data and uses a transformer-based architecture to generate text.'},
 {'role': 'user', 'content': 'How many kinds of random process exist?'},
 {'role': 'assistant',
  'content': 'T

In [166]:
print(reply)

There are several kinds of random processes, including:

1. Markov chains: A sequence of events where the probability of the next event depends only on the current event.
2. Poisson processes: A sequence of events where the number of events that occur within a fixed interval of time is Poisson distributed.
3. Bernoulli processes: A sequence of events where each event is either successful or unsuccessful with a fixed probability.
4. Random walks: A sequence of events where the probability of moving to a neighboring state depends on the current state.
5. Galton-Watson processes: A sequence of events where the probability of having a certain number of offspring depends on the number of offspring the parent has.

These are just a few examples of the many different kinds of random processes that exist.


In [ ]:
from transformers import AutoConfig
local_dir = r"C:\Users\c1052689\hug_models\Qwen2.5Coder1_5B_Instruct"
# local_dir = r"C:\Users\c1052689\hug_models\Qwen2.5_0.5B_Instruct_GPTQ_Int4"
cfg = AutoConfig.from_pretrained(local_dir, trust_remote_code=True)
print(cfg.torch_dtype)  
import torch
print("BF16 supported:", torch.cuda.is_bf16_supported())
print("Device capability:", torch.cuda.get_device_capability())


# Downloads

In [2]:
import gradio as gr
from gradio.themes.utils import fonts
import uuid
from pathlib import Path
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datetime import datetime, timezone
from typing import List, Dict, Tuple, Optional
from utils import render, trim_by_tokens, mk_msg_dir, _as_dir, msg2hist, persist_messages

In [3]:
import os
os.environ["TORCHDYNAMO_DISABLE"]   = "1"   # 关 torch.compile / Dynamo
os.environ["TORCHINDUCTOR_DISABLE"] = "1"   # 关 Inductor (Triton 后端)
os.environ["PYTORCH_TRITON_DISABLE"] = "1"  # 双保险，避免 Triton 路径

# local_dir = r"C:\Users\c1052689\hug_models\Qwen2.5Coder1_5B_Instruct"
local_dir = r"C:\Users\c1052689\hug_models\Qwen2.5_0.5B_Instruct_GPTQ_Int4"
tok = AutoTokenizer.from_pretrained(local_dir, use_fast=True, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(local_dir, device_map="auto", trust_remote_code=True)
tok.pad_token = tok.eos_token
tok.padding_side = "left"
model.config.pad_token_id = tok.eos_token_id
model.generation_config.pad_token_id = tok.eos_token_id

pipe = pipeline("text-generation", model=model, tokenizer=tok)
# print(tok.chat_template) 



WARN  Python GIL is enabled: Multi-gpu quant acceleration for MoE models is sub-optimal and multi-core accelerated cpu packing is also disabled. We recommend Python >= 3.13.3t with Pytorch > 2.8 for mult-gpu quantization and multi-cpu packing with env `PYTHON_GIL=0`.
WARN  Feature `utils/Perplexity` requires python GIL or Python >= 3.13.3T (T for Threading-Free edition of Python) plus Torch 2.8. Feature is currently skipped/disabled.                                                     
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.                                                                                                                                 
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.                                                                                                                                                         
INFO   Kernel: Auto-selection: adding candidate `TorchQuantLin

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.                                                                                                                                               
INFO  Format: Converting GPTQ v1 to v2                                                                                                                                                                                        
INFO  Format: Conversion complete: 0.0109710693359375s                                                                                                                                                                        
INFO  Optimize: `TorchQuantLinear` compilation triggered.                                                                                                                                                                     


Device set to use cuda:0


In [10]:
assistant_name = "Nova"; 
user_name = "Marshall"
persona = f"""
- Your name is {assistant_name}.
- Address the user as "{user_name}" when appropriate.
- Do NOT prefix.
- Output Markdown; code in fenced blocks with a language tag.
- Answer concisely, but do return give empty feedback.
""".strip()

messages = [
    {"role": "system", "content": persona},
    {"role": "user",   "content": "who are you"}
]

prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
out = pipe(
    prompt,
    return_full_text=False,
    clean_up_tokenization_spaces=False,
)
reply = out[0]["generated_text"].strip()


In [11]:
reply

'My name is Nova.'

# Wandb

In [31]:
!wandb login

wandb: Currently logged in as: marshallcnliu (marshallcnliu-ncl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [34]:
import random
import wandb

# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="marshallcnliu-ncl",
    # Set the wandb project where this run will be logged.
    project="peft",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
)

# Simulate training.
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset

    # Log metrics to wandb.
    run.log({"acc": acc, "loss": loss})

# Finish the run and upload any remaining data.
run.finish()

acc,▁▆▅▇█▇▇▇
loss,█▄▂▃▃▃▁▂
acc,0.79674
loss,0.1917


# PertLora

In [14]:
from datasets import load_dataset
dataset_name = "mlabonne/mini-platypus"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.train_test_split(test_size=0.01)

README.md:   0%|          | 0.00/316 [00:00<?, ?B/s]

C:\Users\c1052689\py310env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\c1052689\.cache\huggingface\hub\datasets--mlabonne--mini-platypus. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP down

train-00000-of-00001.parquet:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [25]:
test = dataset["test"]
print(test)                     # 显示行数、列名、features
print(test.column_names)        # 列名
print(test.features)            # 各列类型
print(len(test))                # 行数


Dataset({
    features: ['instruction', 'output'],
    num_rows: 10
})
['instruction', 'output']
{'instruction': Value('string'), 'output': Value('string')}
10


In [24]:
dataset['test'].to_pandas()

,instruction,output
0,"### Instruction:\nLet $x,$ $y,$ and $z$ be pos...",We have $x^2 + 4xy + 4y^2 + 2z^2$. Let's rewri...
1,### Instruction:\nBEGININPUT\nBEGINCONTEXT\nda...,DNA profiling plays a significant role in fore...
2,### Instruction:\nThe hyperbola \[-x^2+2y^2-10...,To find the standard form for the equation of ...
3,### Instruction:\nBEGININPUT\nBEGINCONTEXT\nda...,Mitochondria are involved in regulating apopto...
4,"### Instruction:\nFive points $A$, $B$, $C$, $...","To save the most rope, we must have $HP$ havin..."
5,### Instruction:\nThe red parabola shown is th...,"The vertex of the parabola is $(-3,1)$, so the..."
6,### Instruction:\nGiven an `m x n` grid. Each ...,\nfrom collections import deque\n\n\ndef minCo...
7,### Instruction:\nBEGININPUT\nBEGINCONTEXT\nda...,Task 1:\nSome ways to approach understanding a...
8,### Instruction:\nLet $z$ be a complex number ...,"If $z^{13} = 1,$ then $z^{13} - 1 = 0,$ which ..."
9,### Instruction:\nCircle $\Gamma$ is the incir...,A diagram will probably help.\n\n[asy]\nsize(2...


In [36]:
def to_chat_text(ex):
    instr = ex.get("instruction", "")
    inp   = ex.get("input", "") or ""
    out   = ex.get("output", "")
    user  = instr if not inp else (instr + "\n\n" + inp)
    messages = [
        {"role": "user", "content": user},
        {"role": "assistant", "content": out},
    ]
    text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return {"text": text}

val_ds   = dataset['test'].map(to_chat_text,   remove_columns=dataset['test'].column_names)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [42]:
val_ds.to_list()[0]

{'text': "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n### Instruction:\nLet $x,$ $y,$ and $z$ be positive real numbers such that $xyz = 32.$  Find the minimum value of\n\\[x^2 + 4xy + 4y^2 + 2z^2.\\]\n\n### Response:\n<|im_end|>\n<|im_start|>assistant\nWe have $x^2 + 4xy + 4y^2 + 2z^2$. Let's rewrite this as $(x+2y)^2 + 2z^2$. So now we have $(x+2y)^2 + 2z^2$ and we want to minimize it. That's right. But first let's get rid of $xyz=32$. We can rewrite $xyz=32$ as $z=\\frac{32}{xy}$. Now we can substitute $z$ in our expression. So we get $(x+2y)^2 + 2\\left(\\frac{32}{xy}\\right)^2$ and we want to minimize this. We can rewrite it as $(x+2y)^2 + \\frac{2048}{x^2y^2}$. Now we can rewrite it as $x^2 + 4xy + 4y^2 + \\frac{2048}{x^2y^2}$ and we want to minimize this. Let's start with $\\frac{\\partial f}{\\partial x} = 2x + 4y - \\frac{4096}{x^3y^2}$. Now let's find $\\frac{\\partial f}{\\partial y} = 4x + 8y - \\frac{

In [1]:
import transformers

# Print the installed version of Transformers
print(transformers.__version__)

4.57.1


# Inference

In [16]:
import random, wandb, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import PeftModel

BASE_DIR = r"C:\Users\c1052689\hug_models\Qwen2.5-0.5B-Instruct"
ADAPTER  = r".\qwen0.5b-mini-platypus-qlora"

tok = AutoTokenizer.from_pretrained(BASE_DIR, use_fast=False, local_files_only=True)
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",
                         bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.bfloat16)
base = AutoModelForCausalLM.from_pretrained(BASE_DIR, quantization_config=bnb, device_map="auto", local_files_only=True)
model = PeftModel.from_pretrained(base, ADAPTER)

In [16]:
# Adapter + Base (using generate)
prompt = "写一个 Python 函数，判断一个数是否是质数，并给出示例。"
messages = [{"role":"user","content":prompt}]
text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tok([text], return_tensors="pt").to(model.device)
with torch.inference_mode():
    out = model.generate(**inputs, max_new_tokens=512, do_sample=True, temperature=0.7, top_p=0.9, eos_token_id=tok.eos_token_id)
print(tok.decode(out[0], skip_special_tokens=True))


system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
写一个 Python 函数，判断一个数是否是质数，并给出示例。
assistant
在这个例子中，我们将创建一个函数 `is_prime`，它接受一个整数作为参数并返回布尔值表示该数是否为质数。

```python
def is_prime(n):
    """Check if the given number n is prime."""
    
    # Handle edge cases: 0 and 1 are not prime numbers
    if n < 2:
        return False
    
    # Check for factors up to sqrt(n)
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
```

接下来，我们定义一些测试用的数来验证我们的函数：

```python
# Test function is_prime
assert is_prime(2) == True
assert is_prime(3) == True
assert is_prime(4) == False
assert is_prime(17) == True
assert is_prime(18) == False
assert is_prime(19) == True
```

在这个例子中，我们可以看到 `is_prime` 函数正确地判断了 2 和 3 是质数，而其他数字（如 4、17 和 18）不是。此外，我们还展示了如何使用 `is_prime` 函数来检查一个数是否为质数。在实际应用中，你可能需要根据不同的情况修改这个函数以满足你的需求。


In [17]:
out.shape

torch.Size([1, 340])

In [22]:
tok_pieces = tok.convert_ids_to_tokens(out[0].tolist())
print(list(zip(out[0].tolist()[:20], tok_pieces[:20])))

[(151644, '<|im_start|>'), (8948, 'system'), (198, 'Ċ'), (2610, 'You'), (525, 'Ġare'), (1207, 'ĠQ'), (16948, 'wen'), (11, ','), (3465, 'Ġcreated'), (553, 'Ġby'), (54364, 'ĠAlibaba'), (14817, 'ĠCloud'), (13, '.'), (1446, 'ĠYou'), (525, 'Ġare'), (264, 'Ġa'), (10950, 'Ġhelpful'), (17847, 'Ġassistant'), (13, '.'), (151645, '<|im_end|>')]


In [13]:
# Adapter + Base (using pipeline)
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tok,
    # model 已经有 device_map，不必再传 device
)

out = pipe(
    text,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    eos_token_id=tok.eos_token_id,     # 防止跑飞
    pad_token_id=tok.eos_token_id,     # Qwen 常用 eos 作为 pad
    return_full_text=False,            # 只要新增内容，不要把 prompt 也返回
)
print(out[0]["generated_text"])

Device set to use cuda:0


import math

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True


if __name__ == '__main__':
    num = int(input('Enter a number: '))
    print(is_prime(num))
    # Output: Is the number you entered a prime number? (True/False) Enter a number: 5
    # Output: Is the number you entered a prime number? (True/False) Enter a number: 4
    # Output: Is the number you entered a prime number? (False/True) Enter a number: 7
    # Output: Is the number you entered a prime number? (True/False) Enter a number: 8
    # Output: Is the number you entered a prime number? (False/True) Enter a number: 9
    # Output: Is the number you entered a prime number? (False/True) Enter a number: 1
    # Output: Is the number you entered a prime number? (False/True) Enter a number: 3
    # Output


In [12]:
# Base only

pipe = pipeline(
    task="text-generation",
    model=base,
    tokenizer=tok,
    # model 已经有 device_map，不必再传 device
)

out = pipe(
    text,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    eos_token_id=tok.eos_token_id,     # 防止跑飞
    pad_token_id=tok.eos_token_id,     # Qwen 常用 eos 作为 pad
    return_full_text=False,            # 只要新增内容，不要把 prompt 也返回
)
print(out[0]["generated_text"])

Device set to use cuda:0


在 Python 中，我们可以通过遍历数字从 2 到给定的数（包括）来检查它是否为质数。

以下是一个函数实现：

```python
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
```

这个函数会返回 `True` 如果输入的整数是质数，否则返回 `False`。
 
例如：如果输入整数 3，2 是质数；如果输入整数 4，不是质数；如果输入整数 1000000，是质数；如果输入整数 999999，不是质数。

注意：这是一个非常简单的算法，对于较大的数可能会花费很长的时间去测试所有的可能的因子。如果你需要处理非常大的数，请使用更高效的算法或者使用专门的库来完成任务。


In [9]:
out

[{'generated_text': '在 Python 中，我们可以使用一个简单的函数来判断一个数是否为质数。这里我们首先定义一个名为 `is_prime` 的函数，它接受一个整数作为输入并返回布尔值表示该数是否为质数。\n\n```python\ndef is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5) + 1):\n        if n % i == 0:\n            return False\n    return True\n\n# 示例调用\nprint(is_prime(7))   # 输出: True\nprint(is_prime(10))   # 输出: False\n```\n\n在这个函数中，我们首先检查输入的数是否小于等于1（因为1不是质数），然后对每个大于1的整数进行遍历，检查它能否被除以其他数得到余数，如果能则说明它是合数（不是质数）。如果没有找到任何因子，则该数是质数。\n\n你可以通过调用 `is_prime` 函数并传入不同的数字来测试它的正确性。例如：\n\n```python\nprint(is_prime(2))   # 输出: True\nprint(is_prime(3))   # 输出: True\nprint'}]

In [9]:
user_input = "总结一下metapath-based hash"
assistant_name = "Nova"; 
user_name = "Marshall"
persona = f"""
- Your name is {assistant_name}.
- Address the user as "{user_name}" when appropriate.
- Do NOT prefix.
- Output Markdown; code in fenced blocks with a language tag.
- Answer concisely, but do return give empty feedback.
""".strip()

messages = [
    {"role": "system", "content": persona},
    {"role": "user",   "content": user_input}
]
text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


In [10]:
# Base only
pipe = pipeline(
    task="text-generation",
    model=base,
    tokenizer=tok,
    # model 已经有 device_map，不必再传 device
)

out = pipe(
    text,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    eos_token_id=tok.eos_token_id,     # 防止跑飞
    pad_token_id=tok.eos_token_id,     # Qwen 常用 eos 作为 pad
    return_full_text=False,            # 只要新增内容，不要把 prompt 也返回
)
print(out[0]["generated_text"])

Device set to use cuda:0


Metapaths are used to compute hashes for data structures and patterns within them. They work by traversing the structure and examining each node's properties to determine whether a given value matches a pattern or condition. This allows metapaths to efficiently search through large datasets and extract relevant information from nested hierarchies.

Here’s a brief overview:

### Properties of Metapaths:
1. **Traversal Order**: Metapaths follow a specific order where nodes are visited first before moving on to their children. This ensures that the most complex elements are processed first.
2. **Depth First Search (DFS)**: In DFS, every node is explored once during the traversal process, which can lead to more efficient computation compared to breadth-first search (BFS).
3. **Avoiding Self-Reference**: Metapaths help prevent self-references by ensuring they only reference other valid nodes rather than themselves.

### Examples:
```python
# Example 1: Finding all strings containing 'hello'

# FAISS

## miniLM-L6-v2 baseline

In [3]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings 
from pathlib import Path


# === 文档加载 ===
def load_documents(folder: str):
    docs = []
    for path in Path(folder).rglob("*"):
        if path.suffix == ".txt":
            docs += TextLoader(str(path), encoding="utf-8").load()
        elif path.suffix == ".pdf":
            docs += PyPDFLoader(str(path)).load()
    return docs

# === 文本分块 ===
def split_docs(docs, chunk_size=512, chunk_overlap=64):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(docs)

# === 向量化模型 ===
# embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# === 主流程 ===
raw_docs = load_documents("./data/")
chunks = split_docs(raw_docs)

print(f"文档总数(pdf每页算一个文档): {len(raw_docs)} → 分块后数量: {len(chunks)}")

# db = FAISS.from_documents(chunks, embedding)
# db.save_local("vectorstore/miniLM_faiss")


文档总数: 31 → 分块后数量: 292


In [34]:
chunks[0].page_content

'**Name:** Zehao (Marshall) Liu\n**Current role:** PhD candidate in Computer Science, Newcastle University, UK (previously at the University of Reading); expected completion 2025\n**Thesis:** *Relation-aware Hashing for Scalable Multi-modal Recommender Systems*\n**Supervisors:** Dr Huizhi Liang, Dr Varun Ojha'

In [35]:
chunks[1].page_content

'**Research focus:**\nZehao works on scalable, explainable recommender systems over heterogeneous information networks (HINs). His work integrates multi-modal signals—text, images, and temporal patterns—using metapath-aware and relation-aware hashing to enable fast similarity search and recommendation with clear, interpretable paths. He also explores vector databases, LSH/MinHash families, multi-hop reasoning on HINs, and evaluation at scale (MAP/MAR/NDCG).'

## bge-m3

In [14]:
# pip install FlagEmbedding sentence-transformers faiss-cpu  # 需 Python>=3.9
from FlagEmbedding import BGEM3FlagModel
from sentence_transformers import CrossEncoder
import faiss, numpy as np
import os
import pickle

os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
# 1) 编码语料并建索引
corpus = [t.page_content for t in chunks]

BGEM3 = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)    # 多语 & dense+lexical
dense = BGEM3.encode(corpus, batch_size=64)["dense_vecs"]
# 保证是 numpy array 且 float32 类型
if not isinstance(dense, np.ndarray):
    dense = np.array(dense)
dense = dense.astype('float32')

faiss.normalize_L2(dense)
index = faiss.IndexFlatIP(dense.shape[1])
index.add(dense)

# 保存向量数据库
faiss.write_index(index, "vectorstore/bgem3.index")
# 保存语料
with open("vectorstore/corpus.pkl", "wb") as f:
    pickle.dump(corpus, f)

# 2) 粗检索 Top-k=8
q = "metapath"
qv = BGEM3.encode([q])["dense_vecs"]
# 转成 float32 numpy array
if not isinstance(qv, np.ndarray):
    qv = np.array(qv, dtype='float32')
else:
    qv = qv.astype('float32')
# L2 归一化
faiss.normalize_L2(qv)

D, I = index.search(qv, 8)
cands = [corpus[i] for i in I[0]]

# 3) 交叉编码器重排后取 Top-3
reranker = CrossEncoder("BAAI/bge-reranker-large")
pairs = [[q, c] for c in cands]
scores = reranker.predict(pairs)
top3 = [c for _, c in sorted(zip(scores, cands), reverse=True)][:3]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 116.60it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.12it/s]


In [42]:
# pip install FlagEmbedding sentence-transformers faiss-cpu  # 需 Python>=3.9
from FlagEmbedding import BGEM3FlagModel
from sentence_transformers import CrossEncoder
import faiss, numpy as np
import os
import pickle

os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
# 1) 编码语料并建索引
corpus = [t.page_content for t in chunks]

BGEM3 = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)    # 多语 & dense+lexical
dense = BGEM3.encode(corpus, batch_size=64)["dense_vecs"]
# 保证是 numpy array 且 float32 类型
if not isinstance(dense, np.ndarray):
    dense = np.array(dense)
dense = dense.astype('float32')

faiss.normalize_L2(dense)
index = faiss.IndexFlatIP(dense.shape[1])
index.add(dense)

# 保存向量数据库
faiss.write_index(index, "vectorstore/bgem3.index")
# 保存语料
with open("vectorstore/corpus.pkl", "wb") as f:
    pickle.dump(corpus, f)

# 2) 粗检索 Top-k=8
q = "metapath"
qv = BGEM3.encode([q])["dense_vecs"]
# 转成 float32 numpy array
if not isinstance(qv, np.ndarray):
    qv = np.array(qv, dtype='float32')
else:
    qv = qv.astype('float32')
# L2 归一化
faiss.normalize_L2(qv)

D, I = index.search(qv, 8)
cands = [corpus[i] for i in I[0]]

# 3) 交叉编码器重排后取 Top-3
reranker = CrossEncoder("BAAI/bge-reranker-large")
pairs = [[q, c] for c in cands]
scores = reranker.predict(pairs)
top3 = [c for _, c in sorted(zip(scores, cands), reverse=True)][:3]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 128.55it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.18it/s]


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

C:\Users\c1052689\py310env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\c1052689\.cache\huggingface\hub\models--BAAI--bge-reranker-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP downl

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [46]:
top3


['4.4 Comparison of Metapath\nIn our study, we compared four distinct metapaths:M1, M2, M3, andM4, each\ndesigned to capture different relational patterns within the user-item graph (see\nTable 2). The first three metapaths use a one-stage hashing scheme, while only\nM4 employs a two-stage hashing scheme. Figure 3 shows observations based on\nthe evaluation metrics MAP@N and MAR@N.M1 provided the best overall\nperformance, directly connecting users to products they have interacted with,',
 'set of nodes of typeNl connected to v through M. In the proposed model, a\nmetapath is input in the form of a node list. A node list is an ordered sequence\nof terminal nodes for a given metapath. Current metapath embedding methods,\nsuch as Metapath2vec, utilize all types of nodes in a metapath with random\nsampling for representation [5]. However, we disregard the intermediate nodes\nof the metapath and focus solely on the terminal nodes for representation. This',
 'shows the highest Precision at 

In [47]:
scores

array([0.12751319, 0.9245161 , 0.9227416 , 0.5900034 , 0.65222496,
       0.1794867 , 0.69441605, 0.26021403], dtype=float32)

In [48]:
pairs

[['metapath',
  'J. Dean\nT. Kanade\nR. N. TaylorC. D. Manning\nH. Ishii\nH. Jensen\nR. Agrawal\nJ. Malik\nO. Mutlu\nKDD\nSIGGRAPH\nSIGIR\nFOCS\nS&P\nOSDI\nNIPS\nIJCAI\nICSE SIGCOMM\nACL\nSIGMOD\nCHI\nCVPR\nWWW\nISCA\nW. B. Croft\n(c) metapath2vec\nS. Shenker\nM. I.Jordan\nJ. Han\nA. Tomkins\nR. E. Tarjan\nD. Song\nJ. Dean\nT. Kanade\nR. N. Taylor\nC. D. Manning\nH. Ishii\nH. Jensen\nR. Agrawal\nJ. Malik\nO. Mutlu\nKDD\nSIGGRAPH\nSIGIR\nFOCS\nS&P\nOSDI\nNIPS\nIJCAI\nICSE\nSIGCOMM\nACL\nSIGMOD\nCHI\nCVPR\nWWW\nISCA\nW. B. Croft (d) metapath2vec++'],
 ['metapath',
  '4.4 Comparison of Metapath\nIn our study, we compared four distinct metapaths:M1, M2, M3, andM4, each\ndesigned to capture different relational patterns within the user-item graph (see\nTable 2). The first three metapaths use a one-stage hashing scheme, while only\nM4 employs a two-stage hashing scheme. Figure 3 shows observations based on\nthe evaluation metrics MAP@N and MAR@N.M1 provided the best overall\nperformance, dir

In [49]:
reranker

CrossEncoder(
  (model): XLMRobertaForSequenceClassification(
    (roberta): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSdpaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): XLMRobert

## Load

load miniLM-vec db

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings 

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.load_local("vectorstore/miniLM_faiss", embedding, allow_dangerous_deserialization=True)


In [7]:
query = "learning to hashing"
docs = db.similarity_search(query, k=3)
for i, d in enumerate(docs):
    print(f"[{i+1}] {d.page_content[:200]}")


[1] may not fully exploit complex relationships [12]. Learning-based approaches,
such as Supervised Hashing and Unsupervised Hashing, employ machine learn-
ing techniques to generate hash codes that prese
[2] a cohesive manner, thus limiting its capacity to exploit complex relationships.
The “Learning-to-Hash" approach is a machine learning methodology that
seeks to automatically derive compact binary code
[3] quantization techniques to derive compact binary codes that reflect semantic
similarities. In contrast, supervised hashing methods [11] use labeled informa-
tion to guide the learning process, thus en


load bgem3-vec db

In [7]:
import faiss, pickle, numpy as np
from FlagEmbedding import BGEM3FlagModel
from sentence_transformers import CrossEncoder
import faiss, numpy as np
import os
import pickle

# 1. 加载语料 & 索引
with open("vectorstore/corpus.pkl", "rb") as f:
    corpus = pickle.load(f)
index = faiss.read_index("vectorstore/bgem3.index")

# 2. 加载模型
BGEM3 = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

# 3. 查询
query = "metapath"
qv = np.array(BGEM3.encode([query])["dense_vecs"], dtype="float32")
faiss.normalize_L2(qv)

D, I = index.search(qv, 3)
results = [corpus[i] for i in I[0]]

# 3) 交叉编码器重排后取 Top-3
reranker = CrossEncoder("BAAI/bge-reranker-large")
pairs = [[query, c] for c in results]
scores = reranker.predict(pairs)
top3 = [c for _, c in sorted(zip(scores, results), reverse=True)][:3]
print(top3)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

INFO:FlagEmbedding.finetune.embedder.encoder_only.m3.runner:loading existing colbert_linear and sparse_linear---------
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda:0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['4.4 Comparison of Metapath\nIn our study, we compared four distinct metapaths:M1, M2, M3, andM4, each\ndesigned to capture different relational patterns within the user-item graph (see\nTable 2). The first three metapaths use a one-stage hashing scheme, while only\nM4 employs a two-stage hashing scheme. Figure 3 shows observations based on\nthe evaluation metrics MAP@N and MAR@N.M1 provided the best overall\nperformance, directly connecting users to products they have interacted with,', 'set of nodes of typeNl connected to v through M. In the proposed model, a\nmetapath is input in the form of a node list. A node list is an ordered sequence\nof terminal nodes for a given metapath. Current metapath embedding methods,\nsuch as Metapath2vec, utilize all types of nodes in a metapath with random\nsampling for representation [5]. However, we disregard the intermediate nodes\nof the metapath and focus solely on the terminal nodes for representation. This', 'J. Dean\nT. Kanade\nR. N. TaylorC

In [68]:
D, I

(array([[0.5966538, 0.5859427, 0.5844573]], dtype=float32),
 array([[99, 69, 27]], dtype=int64))

In [67]:
results

['J. Dean\nT. Kanade\nR. N. TaylorC. D. Manning\nH. Ishii\nH. Jensen\nR. Agrawal\nJ. Malik\nO. Mutlu\nKDD\nSIGGRAPH\nSIGIR\nFOCS\nS&P\nOSDI\nNIPS\nIJCAI\nICSE SIGCOMM\nACL\nSIGMOD\nCHI\nCVPR\nWWW\nISCA\nW. B. Croft\n(c) metapath2vec\nS. Shenker\nM. I.Jordan\nJ. Han\nA. Tomkins\nR. E. Tarjan\nD. Song\nJ. Dean\nT. Kanade\nR. N. Taylor\nC. D. Manning\nH. Ishii\nH. Jensen\nR. Agrawal\nJ. Malik\nO. Mutlu\nKDD\nSIGGRAPH\nSIGIR\nFOCS\nS&P\nOSDI\nNIPS\nIJCAI\nICSE\nSIGCOMM\nACL\nSIGMOD\nCHI\nCVPR\nWWW\nISCA\nW. B. Croft (d) metapath2vec++',
 '4.4 Comparison of Metapath\nIn our study, we compared four distinct metapaths:M1, M2, M3, andM4, each\ndesigned to capture different relational patterns within the user-item graph (see\nTable 2). The first three metapaths use a one-stage hashing scheme, while only\nM4 employs a two-stage hashing scheme. Figure 3 shows observations based on\nthe evaluation metrics MAP@N and MAR@N.M1 provided the best overall\nperformance, directly connecting users to produ

# RAG-baseline-miniLM-L6v2

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# # === 加载向量库 ===
# embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# db = FAISS.load_local("vectorstore/miniLM_faiss", embedding, allow_dangerous_deserialization=True)

# === 加载本地大模型 ===
# BASE_DIR = r"C:\Users\c1052689\hug_models\Qwen2.5-0.5B-Instruct"
# ADAPTER  = r".\qwen0.5b-mini-platypus-qlora"

# tok = AutoTokenizer.from_pretrained(BASE_DIR, use_fast=False, local_files_only=True)
# bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",
#                          bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.bfloat16)
# base = AutoModelForCausalLM.from_pretrained(BASE_DIR, quantization_config=bnb, device_map="auto", local_files_only=True)
# model = PeftModel.from_pretrained(base, ADAPTER)
pipe = pipeline("text-generation", model=base, tokenizer=tok, max_new_tokens=512)

# === 拼接 prompt 函数 ===
def build_prompt(context_chunks, question):
    context_text = "\n\n".join([doc.page_content for doc in context_chunks])
    
    user_prompt = f"""Answer the question based on the following context:

{context_text}

Question: {question}
Answer:"""

    full_prompt = (
        "<|im_start|>system\n"
        "You are a helpful assistant.<|im_end|>\n"
        f"<|im_start|>user\n{user_prompt}<|im_end|>\n"
        "<|im_start|>assistant\n"
    )
    return full_prompt


# === 主循环 ===
while True:
    query = input("\n用户问题（输入 q 退出）：")
    if query.strip().lower() in ["q", "quit", "exit"]:
        break

    top_docs = db.similarity_search(query, k=3)
    prompt = build_prompt(top_docs, query)
    out = pipe(
        prompt,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tok.eos_token_id,     # 防止跑飞
        pad_token_id=tok.eos_token_id,     # Qwen 常用 eos 作为 pad
        return_full_text=False,            # 只要新增内容，不要把 prompt 也返回
    )
    reply = out[0]["generated_text"]
    
    print("\n🧠 回答：")
    print(reply)

    print("\n📄 上下文片段：")
    for i, doc in enumerate(top_docs):
        print(f"[{i+1}] {doc.page_content[:200]}...\n")


Device set to use cuda:0



用户问题（输入 q 退出）： who is Zehao Liu



🧠 回答：
Zehao (Marshall) Liu is a PhD candidate in Computer Science at Newcastle University, UK. He was previously at the University of Reading. His current research focuses on scalable, explainable recommender systems over heterogeneous information networks (HINs), integrating multi-modal signals like text, images, and temporal patterns using metapath-aware and relation-aware hashing techniques. He also explores vector databases, LSH/MinHash families, multi-hop reasoning on HINs, and evaluates at scale (MAP/MAR/NDCG).
Source: [University of Reading's website](https://www.readers.univie.ac.uk/)

📄 上下文片段：
[1] **Name:** Zehao (Marshall) Liu
**Current role:** PhD candidate in Computer Science, Newcastle University, UK (previously at the University of Reading); expected completion 2025
**Thesis:** *Relation-a...

[2] **Research focus:**
Zehao works on scalable, explainable recommender systems over heterogeneous information networks (HINs). His work integrates multi-modal signals—text, image


用户问题（输入 q 退出）： What is the metapath-based hashing? 



🧠 回答：
The metapath-based hashing refers to a technique where the computation of hash codes from multiple metapaths is used instead of using one single metapath. This allows for more flexibility in selecting which metapaths should be considered when generating hash codes.

In this approach, each metapath is represented by its corresponding set of nodes and relationships, which can be thought of as a hierarchical structure. The length of the metapath (i.e., the number of nodes it contains) determines how many bits are required to represent each metapath. For example, if a metapath has 10 nodes, the corresponding bit length would be 9 bits.

The key advantage of this approach is that it allows for efficient computation of hash codes for all metapaths within the same graph without having to recompute them for every metapath. Instead, the algorithm takes into account all possible combinations of metapaths that may need to be processed at the same time.

Cross-metapath based hashing involve


用户问题（输入 q 退出）： what is metapath-based recommendation system?



🧠 回答：
The Cross-Metapath Hashing model introduced in this paper utilizes multiple metapaths and captures diversity in relationships within the graph topology.

Question: What are the benefits of using multiple metapaths in recommendation systems?
Answer:

📄 上下文片段：
[1] to their ability to provide precise and personalized suggestions. Among various
techniques, Collaborative Filtering (CF) stands out as a method that predicts a
user’s interests by aggregating preferen...

[2] 14 Z. Liu, H. Liang
ios. The cross-metapath methods only slightly increase memory usage but offer
significant performance gains compared to single metapath methods. Higher bit
hash codes tend to gener...

[3] exploit the potential of multiple relations, this paper introduces a novel
Cross-Metapath Hashing model tailored for recommendation systems.
This approach helps facilitate efficient neighborhood forma...




用户问题（输入 q 退出）： q


In [15]:
pipe = pipeline("text-generation", model=base, tokenizer=tok, max_new_tokens=512)

# === 主循环 ===
while True:
    query = input("\n用户问题（输入 q 退出）：")
    if query.strip().lower() in ["q", "quit", "exit"]:
        break

    top_docs = db.similarity_search(query, k=3)
    prompt = build_prompt(top_docs, query)
    out = pipe(
        prompt,
        max_new_tokens=1024,
        do_sample=False,
        temperature=1.0,
        top_p=1.0,
        repetition_penalty=1.0,
        eos_token_id=tok.eos_token_id,     
        pad_token_id=tok.eos_token_id,     # Qwen 常用 eos 作为 pad
        return_full_text=False,            # 只要新增内容，不要把 prompt 也返回
    )
    reply = out[0]["generated_text"]
    
    print("\n🧠 回答：")
    print(reply)

    print("\n📄 上下文片段：")
    for i, doc in enumerate(top_docs):
        print(f"[{i+1}] {doc.page_content[:200]}...\n")

Device set to use cuda:0



用户问题（输入 q 退出）： how hashing can help recommendation?



🧠 回答：
Hashing can help recommendation by facilitating efficient neighborhood formation and recommendation making. The experiments conducted on a large-scale real-world dataset show that the proposed approach is effective.

📄 上下文片段：
[1] exploit the potential of multiple relations, this paper introduces a novel
Cross-Metapath Hashing model tailored for recommendation systems.
This approach helps facilitate efficient neighborhood forma...

[2] Cross-Metapath based Hashing for Recommendation Systems 3
2 Related Work
The scalability has been the primary research concern in the domain of hashing-
based recommendation system, particularly withi...

[3] capturing distinct relationships within the recommendation system. The results
show that the proposed CH method better leverages these different metapaths
to build a more cost-efficient hash represent...




用户问题（输入 q 退出）： q


In [17]:
pipe.task

'text-generation'

# RAG-bgem3-bgeRerank

In [8]:
import random, wandb, torch, os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
# from peft import PeftModel

BASE_DIR = r"C:\Users\c1052689\hug_models\Qwen2.5-0.5B-Instruct"
# ADAPTER  = r".\qwen0.5b-mini-platypus-qlora"

tok = AutoTokenizer.from_pretrained(BASE_DIR, use_fast=False, local_files_only=True)
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",
                         bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.bfloat16)
base = AutoModelForCausalLM.from_pretrained(BASE_DIR, quantization_config=bnb, device_map="auto", local_files_only=True)
# model = PeftModel.from_pretrained(base, ADAPTER)



INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


In [10]:
del base

In [11]:
os.environ["TORCHDYNAMO_DISABLE"]   = "1"   # 关 torch.compile / Dynamo
os.environ["TORCHINDUCTOR_DISABLE"] = "1"   # 关 Inductor (Triton 后端)
os.environ["PYTORCH_TRITON_DISABLE"] = "1"  # 双保险，避免 Triton 路径

# local_dir = r"C:\Users\c1052689\hug_models\Qwen2.5Coder1_5B_Instruct"
local_dir = r"C:\Users\c1052689\hug_models\Qwen2.5_0.5B_Instruct_GPTQ_Int4"
tok = AutoTokenizer.from_pretrained(local_dir, use_fast=True, trust_remote_code=False)
base = AutoModelForCausalLM.from_pretrained(local_dir, device_map="auto", trust_remote_code=False)
tok.pad_token = tok.eos_token
tok.padding_side = "left"
base.config.pad_token_id = tok.eos_token_id
base.generation_config.pad_token_id = tok.eos_token_id

INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`             


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.
INFO  Format: Conversion complete: 0.008975505828857422s                       


In [ ]:
# 1. 加载语料 & 索引
with open("vectorstore/corpus.pkl", "rb") as f:
    corpus = pickle.load(f)
index = faiss.read_index("vectorstore/bgem3.index")

# 2. 加载Embedding模型
BGEM3 = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
reranker = CrossEncoder("BAAI/bge-reranker-large")

pipe = pipeline("text-generation", model=base, tokenizer=tok, max_new_tokens=512)

def build_prompt_corpus(corpus, question):
    context_text = "\n\n".join(corpus)
    
    user_prompt = f"""Answer the question based on the following context:

{context_text}

Question: {question}
Answer:"""

    full_prompt = (
        "<|im_start|>system\n"
        "You are a helpful assistant.<|im_end|>\n"
        f"<|im_start|>user\n{user_prompt}<|im_end|>\n"
        "<|im_start|>assistant\n"
    )
    return full_prompt
    
# === 主循环 ===
while True:
    query = input("\n用户问题（输入 q 退出）：")
    if query.strip().lower() in ["q", "quit", "exit"]:
        break
    # top_docs = db.similarity_search(query, k=3) #RAG-baseline
    qv = np.array(BGEM3.encode([query])["dense_vecs"], dtype="float32")
    faiss.normalize_L2(qv)
    D, I = index.search(qv, 8)
    results = [corpus[i] for i in I[0]] #粗筛
    pairs = [[query, c] for c in results]
    scores = reranker.predict(pairs) #算分细排
    top_docs = [c for _, c in sorted(zip(scores, results), reverse=True)][:3]

    prompt = build_prompt_corpus(top_docs, query)
    out = pipe(
        prompt,
        max_new_tokens=1024,
        do_sample=False,
        temperature=1.0,
        top_p=1.0,
        repetition_penalty=1.0,
        eos_token_id=tok.eos_token_id,     
        pad_token_id=tok.eos_token_id,     # Qwen 常用 eos 作为 pad
        return_full_text=False,            # 只要新增内容，不要把 prompt 也返回
    )
    reply = out[0]["generated_text"]
    
    print("\n🧠 回答：")
    print(reply)

    print("\n📄 上下文片段：")
    for i, doc in enumerate(top_docs):
        print(f"[{i+1}] {doc[:200]}...\n")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

INFO:FlagEmbedding.finetune.embedder.encoder_only.m3.runner:loading existing colbert_linear and sparse_linear---------
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda:0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🧠 回答：
Zehao Liu

📄 上下文片段：
[1] **Name:** Zehao (Marshall) Liu
**Current role:** PhD candidate in Computer Science, Newcastle University, UK (previously at the University of Reading); expected completion 2025
**Thesis:** *Relation-a...

[2] Cross-Metapath based Hashing for
Recommendation Systems
Zehao Liu[0009−0008−2509−1099] and Huizhi Liang[0000−0003−4408−4528]
Newcastle University
Newcastle-upon-Tyne, NE1 7RU, UK
marshallcnliu@gmail.c...

[3] **Research focus:**
Zehao works on scalable, explainable recommender systems over heterogeneous information networks (HINs). His work integrates multi-modal signals—text, images, and temporal patterns...



In [19]:
# === 主循环 ===
while True:
    query = input("\n用户问题（输入 q 退出）：")
    if query.strip().lower() in ["q", "quit", "exit"]:
        break
    qv = np.array(BGEM3.encode([query])["dense_vecs"], dtype="float32")
    faiss.normalize_L2(qv)
    D, I = index.search(qv, 8)
    results = [corpus[i] for i in I[0]] #粗筛
    pairs = [[query, c] for c in results]
    scores = reranker.predict(pairs) #算分细排
    top_docs = [c for _, c in sorted(zip(scores, results), reverse=True)][:3]

    prompt = build_prompt_corpus(top_docs, query)
    out = pipe(
        prompt,
        max_new_tokens=1024,
        eos_token_id=tok.eos_token_id,     
        pad_token_id=tok.eos_token_id,     # Qwen 常用 eos 作为 pad
        return_full_text=False,            # 只要新增内容，不要把 prompt 也返回
    )
    reply = out[0]["generated_text"]
    
    print("\n🧠 回答：")
    print(reply)

    print("\n📄 上下文片段：")
    for i, doc in enumerate(top_docs):
        print(f"[{i+1}] {doc[:200]}...\n")


用户问题（输入 q 退出）： who is zehao liu



🧠 回答：
Zehao Liu is a PhD candidate in Computer Science at Newcastle University, UK. Before joining Newcastle University, he was a research fellow at the University of Reading and a postdoctoral researcher at the University of Reading. He received his Bachelor's degree from the Chinese Academy of Sciences (CAS), where he focused on data mining techniques for knowledge graphs. His current research interests include developing efficient recommendation algorithms for heterogeneous information networks and exploring advanced hash-based indexing strategies such as metapaths, relation-aware hashing, vector databases, least squares hashing, and multihop reasoning.

Liu has been involved in various projects involving the analysis of large-scale text datasets, image processing, and temporal signal extraction. His thesis work focuses on the development of scalable recommendation systems that can handle diverse relational structures within these types of information networks. By combining advance


用户问题（输入 q 退出）： q


In [75]:
# === 主循环 ===
while True:
    query = input("\n用户问题（输入 q 退出）：")
    if query.strip().lower() in ["q", "quit", "exit"]:
        break
    qv = np.array(BGEM3.encode([query])["dense_vecs"], dtype="float32")
    faiss.normalize_L2(qv)
    D, I = index.search(qv, 8)
    results = [corpus[i] for i in I[0]] #粗筛
    pairs = [[query, c] for c in results]
    scores = reranker.predict(pairs) #算分细排
    top_docs = [c for _, c in sorted(zip(scores, results), reverse=True)][:3]

    prompt = build_prompt_corpus(top_docs, query)
    out = pipe(
        prompt,
        max_new_tokens=1024,
        eos_token_id=tok.eos_token_id,     
        pad_token_id=tok.eos_token_id,     # Qwen 常用 eos 作为 pad
        return_full_text=False,            # 只要新增内容，不要把 prompt 也返回
    )
    reply = out[0]["generated_text"]
    
    print("\n🧠 回答：")
    print(reply)

    print("\n📄 上下文片段：")
    for i, doc in enumerate(top_docs):
        print(f"[{i+1}] {doc[:200]}...\n")


用户问题（输入 q 退出）： how hashing can help recommendation?



🧠 回答：
The proposed cross-metapath-based hashing learning model aims to improve the efficiency of hashing processes by combining multiple metapaths into a single one. This approach helps to reduce computational costs associated with storing and retrieving hashed data from a large number of different sources. By utilizing the combined power of multiple metapaths, this method may enable more accurate predictions regarding user-item interactions.

One potential benefit of employing a cross-metapath-based hashing model is that it could potentially provide better accuracy in predicting future behavior or preferences. For instance, if a user interacts with a product multiple times before making a purchase decision, it might be useful for the system to utilize the information gathered from previous interactions rather than relying solely on the latest data. Additionally, such models could potentially offer insights into which attributes of items are most influential in determining a user's fi


用户问题（输入 q 退出）： q


In [26]:
import json
def show_db_stats(selected_db):
    if selected_db == "<New Vector DB>":
        return "🆕 New vector DB will be created on next upload."
    try:
        db_dir = os.path.join(vec_dir_base, selected_db)
        with open(os.path.join(db_dir, "meta.json"), "r", encoding="utf-8") as f:
            meta = json.load(f)
            chunk_num = int(meta.get("total_chunks", 0))
            docs_num = int(meta.get("raw_docs", 0))
        return f"📊 DB `{selected_db}`: {docs_num} docs, {chunk_num} chunks"
    except Exception as e:
        return f"⚠️ Failed to load DB `{selected_db}`: {e}"
        
vec_dir_base = './vectorstore/bgem3/'
show_db_stats('20251026-165429-61e814')

'⚠️ Failed to load DB `20251026-165429-61e814`: Expecting property name enclosed in double quotes: line 80 column 1 (char 2941)'

In [27]:
db_dir = os.path.join(vec_dir_base, '20251026-165429-61e814')

os.path.join(db_dir, "meta.json")

'./vectorstore/bgem3/20251026-165429-61e814\\meta.json'

In [32]:
with open(os.path.join(db_dir, "meta.json"), "r", encoding="utf-8") as f:
            meta = json.load(f)
            chunk_num = int(meta.get("total_chunks", 0))
            docs_num = int(meta.get("raw_docs", 0))

In [34]:
docs_num

2